In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import pandas as pd

from keras.datasets import mnist
from keras.utils import np_utils


# Neural Network Basics

We can try to build a neural network from scratch to see the details of how it works. We won't be using these homemade networks for real purposes, but it is good to get a bit of a look at the mechanics to help make things make sense. 

We will start by trying to create to solve a problem that we are familiar with - logistic regression. 

## What is a Neural Network?

Neural networks are a type of machine learning algorithm that is inspired by the way that the human brain works. The human brain is made up of neurons that are connected to each other, and a neural network aims to replicate that. How much this actually models a human brain is up in the air (I don't think it actually works anything like a real human), but it's ability to make predictive models is excellent. We have inputs, output(s), and some number of layers in between. This sounds complex, and it can be, but we can break it down into a few simple steps.

![Neural Network](images/nn_structure.png "Neural Network")

The strucutre of a neural network is based on layers:
<ul>
<li> Input layer - the data that we are feeding into the network. Each of the neurons above gets one variable from the data set.
<li> Output layer - the prediction that the network makes. This generates an actual prediction, just like any of our other models. 
<li> Weights - just like the slopes in regressions, we have weights that are multiplied by the inputs. Also just like regression, these weights are what the model learns to make its predictions.
<li> Middle (hidden) layers - these are the layers that are in between the input and output layers. These layers are where the magic happens and are what allows a neural network to learn about our data so well. 
</ul>

The basic process of training a neural network is also similar to regression:
<ul>
<li> The data comes into the input layer, each is multipled by a weight, and then the results are comined into a prediction. This is called forward propagation.
<li> The model, with the current weights makes a prediction. 
<li> The error of the prediction is calculated.
<li> The errror is broken down and attributed back to each of the weights, in a process called backpropagation. This is just like gradient descent finding adjustments for the weights in a regression, but slighty more complex since we normally have several layers. 
<li> The weights are adjusted based on the error.
<li> A new model with the new weights is made, and the process repeats until the model is good enough or we hit a limit. 
</ul>

![SimpleNN](images/simple_nn.png "SimpleNN" )

This is basically what we looked at in a logistic regression! That is true for a one layer NN, with an activation (more on this later) of a sigmod. We calcualte the weights, make a prediction, use gradient descent to adjust, and filter the output through the sigmoid calcualtion to get a yes/no prediction. The difference here is that we have multiple layers, each with its own set of weights. The more layers we add, the more ability the model has to learn relationships in the data. In a sense (this is a thought process, not a literal description) each layer is kind of similar to its own little logistic regression, and we layer them together like a boosting model to make improvements layer by layer. 

### Helpers - Base Layer and Support Functions

First we need a few helper functions - separating these out will make our code much easier to read. Below we have:
<ul>
<li> Base Layer Class: we are going to have two types of layers - the "normal" fully connected one, and an activation one to apply the activation function. In the future the activation is wrapped into the normal layer, but this will help us see the parts more clearly. 
<li> Activation Functions and Derivitives: we have two activation functions, and the derivitive of each. We'll talk about different activation functions in more detail later on. 
<li> Loss Function and Derivitive: we have the loss function and its derivitive. We'll keep it simple and use MSE, in a real classification this would probably be log-loss. We'll also talk more about different loss functions later on. 
<li> Convert to Bool: this just translates probability predictions [0,1] into a binary classification. We just need it to calculate predicted accuracy of our test data predictions. 
</ul>

In [ ]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

##### Activation Functions

There are a lot of different activation functions that can be used in neural networks, we will define a few that we can use with our simple network. We will focus more on the details of activation functions a bit later on. The activation function allows the network to learn non-linear relationships in the data, and different activation functions may be better for different applications. For now, we can try to swap between a few and observe the results. 

In [ ]:
# activation function and its derivative
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

In [ ]:
# activation function and its derivative
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

def sigmoid_prime(z):
  fz = sigmoid(z)
  return fz * (1 - fz)

In [ ]:
# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

In [ ]:
# Convert a decimal between 0 and 1 to either 0 or 1, based on if it is
# above or below the cutoff. 
def conv_to_bool(float_list, cutoff=.5):
    new_list = []
    for i in float_list:
        if i < cutoff:
            new_list.append(0)
        else:
            new_list.append(1)
    return new_list

### Usable Layers

We can create the layers that we'll actually use now. Normally the FClayer and the activation layer we are creating would be combined, but this makes it easier to explore.

#### Fully Connected Layer (Dense)

The main component of the neural network is the fully connected, or dense, layer. The fully connected (dense) part just means that each neuron is connected to each neuron in the next layer. The three functions each have simple jobs:

<ul>
<li> <b>Initialization:</b> set up the matrix of weights and a vector of bias values. Here they are initialized to a random bumber, in practice these are initialized using some smart method, configurable with a parameter. Each neuron connects to each other neuron, so there is a wight for each of those connections - a matrix of input size by output size. Each output has one bias value. 
<li> <b>Forward Propagation:</b> in forward propagation this layer generates predictions by multiplying weights * values and adding the bias. The calculation itself is a dot product - which multiplies each input by its corresponding weight in the matrix automatically. This could be done by lots of loops, but this is more compact and efficient. Recall that each connection has a weight, so we end up with this large matrix. 
<li> <b>Backward Propagation:</b> this layer calculates the impact of the error stemming from each input X. This is done by calculating the gradient of the loss with respect to each input - thus allowing us to attribute portions of the output error to the different inputs. The calculation is done using the derivitive of the activation function and some math called the chain rule. Once we have this, we update all the weights and the bias so that we shrink our loss. 
    <ul>
    <li> If we look at the backwards propagation code, we can see that the actual math to calculate the gradient is a pretty simple implementation. We can perform the calculation by dot product of the error and the weights, and then multiply by the derivitive of the activation function. This is the gradient of the loss with respect to the input.
    <li> The linear algebra details are a bit beyond what we need to cover, but the important thing is that this calculation is done for the matricies of weights and error, all the way back to the initial layer of weights. Because our neural networks tend to have a large number of neurons, in a large number of layers, this can add up to a lot of caclulations each round of traning - the matrix math that can execute these calculations is something that is optimized for speed in the library code. 
    </ul>
</ul>

![Weights](images/weights.png "Weights" )



In [ ]:
# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        
        # update parameters
        # dBias = output_error
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error

        return input_error

### Activation Functions

One of the most important parts of a neural network is the activation function. The activation function is what allows the network to learn non-linear relationships. Without an activation function, the network would be no more powerful than a linear regression. The activation function is applied to the output of each neuron, and it is what determines whether the neuron is "on" or "off". Prior to the output of each neuron being sent on to the next layer, the activation function is applied. If you're paying close attention, the output going into an activation function is just a linear regression - the sum of the features times their weight. The activation function changes this, and allows the neural network to be "more" than a complicated linear regression. In real applications, we usually have a few layers in our neural network, and the combination of multiple layers and activation functions allows us to learn very complex relationships. The term "deep learning" refers to networks that have lots of layers. 

This step is just like the sigmod step in a logistic regression - we have the raw value that comes into the function, the output is not that raw value, it is whatever that raw value translates to after the activation function is applied. If this wasn't applied, a multi-layer neural network could be decomponsed into a simple linear equation (with a bunch of linear algebra work). As well, each layer has an activation function, and they don't necissarily have to be the same one from layer to layer. In particular, we normally have a different activation function for the last layer of the network, one that will give us the output we want.

![NN Steps](images/nn_steps.jpeg "NN Steps" )

There are many different activation functions, including the one that we are used to - the sigmoid. These activation functions are things that we can experiment with and tune like our hyperparameters. Some tend to be better suited for different types of problems, such as ReLU being common for image work and tanh being common for text processing, but there is no definitive answer. There are also considerations for computational efficiency, and some activation functions are more computationally expensive than others. For the most part we will start with the most common one for the type of problem we are doing, and then experiment with others if we feel the need. We'll look at these more later. If in doubt ReLU, or one of the slight variations on ReLU that we'll introduce later, is a solid choice. 

![Activation Functions](images/activation_functions.png "Activation Functions")

#### Output Activation Functions

At our final layer, the model needs to output an actual prediction. For that final activation function we need to choose one that produces an output that matches what we are prediction. For regression, the final activation function is "nothing" or linear - we want the raw value. For classification, we want a sigmoid for two classes, like a logistic regression, or the softmax for more than two classes. This isn't an absolute rule, but a pretty reliable rule of thumb.

#### Activation Layer

The activation layer is more simple, it just applies the activation function. Normally this is built into the other layer, but this is a little more simple to build by hand. 

<ul>
<li> Initialization: set the activation function and derivitive to use going forward. 
<li> Forward Propagation: in forward propagation the activation layer just takes the input that it gets and applies the activation function to it.  
<li> Backward Propagation: in backward propagation the activation layer translates the error of the predictions back up by multiplying the error by the derivitive of the activation function. This has the effect of translating the error we got with respect to the output of the activation function into error with respect to the input of the activation function. 
</ul>

In [ ]:
# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

## Full Network

We can now build our layers into an actual model. Our code here is a little long, but most of it is pretty simple - the majority of the extra stuff is to make the model flexible. 

<ul>
<li> Add - call this to add layers to the model. 
<li> Use - provide a loss function, and the derivitive of a loss function for the model to use. 
<li> Predict - generate prediction. This is just one execution of a forward propagation for the new inputs. 
<li> Fit - train the model. 
    <ul>
    <li> Each iteration is called an epoch. We loop through the process until we've hit the desired number of epochs. 
    <li> Run each sample through a forward propagation. 
    <li> Get the prediction, and calculate the error. 
    <li> Pass the error to the back propagation. Repeat
    </ul>
</ul>

Another way to phrase the fitting and optimizing of the model is to think of the errors on each epoch. First we generate a prediction (FP), and find the error of that prediction by simply comparing it to the true value - this is the error with respect to y, the output. 

### Forward Propagation

Forward propagation is very simple, we just walk through the network and generate a prediction. We start with the inputs, do the multiplication by the weights, add the bias, and then apply the activation function. This is repeated for each layer all the way through. This is also the .predict() function, we just run the inputs through the network and get the output, like any other model. 

### Back Propagation and Gradient Descent

Next we do the back propagation, this takes that error with respect to y that we figured out, and "breaks" that error down into error with respect to each term from the previous layer. Recall that one neuron creates a prediction (during FP) that is equal to w1*x1 + w2*x2 +... b; this does a sort of reverse execution of that - starting with the overall error and mapping it to each term. The error is effectively split into parts and each of the m*x terms and the b term is labeled as being accountable for some of it. Since the x values are the inputs, they can't change, so we modify the weights and bias to lessen the error - this is the gradient descent part. This gets repeated back through each layer until the first layer, then another FP begins with the new updated weights. The learning rate controls the size of the adjustments at each round.

![Back Propagation](images/backprop.webp "Back Propagation" )

This back propagation part is the key to the high accuracy ceiling that neural networks have when the data is large - we can tune each of many neurons specifically to their contribution of error. If we have enough data, we can get very accurate predictions. This is also the gradient descent process that we are used to from logistic regression. The difference is that we walk that process back through the entire network, through all the layers, and update the weights and bias all the way through. This can become computationally expensive, as we are doing partial derivitives on a bunch of (potentially large) matricies over and over, but the concept is simple - we are adjusting the weights to improve the error, or performing gradient descent.

In [ ]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        errors = []
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                #print(self.loss)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            if i % 100 == 0:
                err /= samples
                errors.append(err)
                print('epoch %d/%d   error=%f' % (i+1, epochs, err))

## Use the Network

We have a neural network now - we need to use it! Since we were smart and made the framework able to handle input data of an arbitrary size, we can use our model for pretty much any application!

<b>Note:</b> for all of these examples, the syntax that we are using is similar to the "real world", but not exactly the same. Our handmade neural network doesn't quite have the same functionality of the libraries we can use, but it is close enough to get the idea. 

### Simple Example - XOR

We can test and see what our network can do with a very simple trial. XOR is a logical operation - exclusive or. It is 1 only if exactly one input is 1, or else it is false. We have that data as the training X and y, we can see if our network can learn this very simple relationship before we test it on some real data. 

![XOR](images/xor.webp "XOR" )

One thing to note here is that the XOR problem can't be solved with a single layer. We need at least two layers to be able to learn the relationship. Let's see if our model can figure it out...

<b>Note:</b> this is a binary classification, so the output layer has a sigmoid activation function. Try to play with a couple of others and see the results. 

In [ ]:
# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 2))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(2, 1))
net.add(ActivationLayer(sigmoid, sigmoid_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=10000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

#### Implement a Logistic Regression

We can create the network that we defined above. You may also recognize this 1 layer model as a logistic regression:
<ul>
<li> The weights are the coefficients of the linear regression.
<li> The bias is the intercept of the linear regression.
<li> The activation function is the sigmoid function.
<li> The features are the features. 
</ul>

At our most simple, that is all a neural network is. The ones that are more capable and flexible are just more complex versions of this, normally with more layers. Each layer adds one of these logistic regression like steps, only they are being performed on the output of the previous layer. Multiple layers allow us to learn more complex relationships, as each layer can learn a different relationship from the transformed data. 

<b>Note:</b> We have to supply the "sigmoid_prime" and similar functions here. That's something that the library would normally do for us, but since we're building it from scratch, we have to do it ourselves. The prime version is what the nerual network uses to calculate the error in back propagation.

In [ ]:
df = pd.read_csv("data/diabetes.csv")
df.head()

### Diabetes Data

One weird thing that we will do to the data here is that we will reshape the Xs into a 3 dimensional array. This is due to the structure of the model - it is expecting data to be in that shape. When we start using Keras (the package we use for neural network models) this can generally be handled automatically, but since we've made this from scratch, we need to do it here. We do need to be a little more comfortable with the shape of the data, and potentially shifting it around, so this is a good exercise.

What we end up with for the data is:
<ul>
<li> 8 numerical features. We will have 8 neurons / Xs in the input of our network. 
<li> 1 binary categorical output. We will have one neuron for the output of the network. 
<li> Activation function applied to our output, to ensure we get categorization. 
</ul>

<b>Note:</b> we always want to scale data for our neural networks. 

![SimpleNN](images/simple_nn.png "SimpleNN" )

In [ ]:
y = np.array(df["Outcome"]).reshape(-1,1)
X = df.drop(columns={"Outcome"})

X_train, X_test, y_train, y_test = train_test_split(X, y)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train).reshape(-1,1,8)
X_test = scaler.transform(X_test).reshape(-1,1,8)

#Look at some details
print ("train_set_x shape: " + str(X_train.shape))
print ("train_set_y shape: " + str(y_train.shape))
print ("test_set_x shape: " + str(X_test.shape))
print ("test_set_y shape: " + str(y_test.shape))
print(X_test[0])
print(y_test[0])
print(X_train[0])
print(y_train[0])

We will make a one layer network, with a simoid activation function. This is a logistic regression, just like my beautiful diagram above. 

In [ ]:
# Network
net = Network()
net.add(FCLayer(8, 1))
net.add(ActivationLayer(sigmoid, sigmoid_prime))

# Train
net.use(mse, mse_prime)
net.fit(X_train, y_train, epochs=1000, learning_rate=0.0001)

# Evaluate on test set
out = net.predict(X_test)
pred_labels = conv_to_bool(out)
print(accuracy_score(y_test, pred_labels))

##### Results

One thing that we can see is that our model doesn't really improve all that much if we let training keep running. This should make some intuitive sense - we are only using one layer, and thus only doing a "regular" gradient descent. Once the model has found good weights, there's not really anything else to adjust, it is kind of just bouncing around near the minimum on the gradient descent curve. The amount that can be learned is pretty limited, and we don't need any more iterations than a regular logistic regression would. 

#### Try a Larger Network

What if we try adding some layers? Outside of the input and output the rest of the stuff inside our network is configurable. We can add whatever we want in terms of layer, and each layer can be any size, as long as the shape matches the previous and next layer. We can also try that other activation function if we want. What's the "right" size? That question doesn't have a direct answer, we'll look at some things we can use to estimate it ~ 2 workbooks from now. We can play around a bit with it now and see what we get. As you can probably gues, larger models tend to be more flexible, more likely to overfit, while smaller models are less flexible, but less capable of learning the complex relationships in data. 

If our data isn't linearly separable, we should see our model more able to fit the data as we start adding layers. Try a few, the only thing that we need to be careful of is that the shapes of inputs and outputs match up. Other than that, we can add as many layers as we want, with different activation functions. 

![MultiLayerNN](images/multi_layer_nn.jpeg "MultiLayerNN" )

In [ ]:
# Network


#### Results

These results will vary depending on exactly what you added for layers, size, and activations, but we will generally see a couple of things as we add some more layers. First, we will see that the model is able to learn the data better. This is because we are adding more layers, and thus more logistic regression steps. Each layer is learning a different relationship from the data, and we are able to learn more complex relationships. Second, we should see more (it might not be a lot, it depends on the setup) learning as we progress through the rounds of training. This is because the capacity of our model to learn is far larger, we have more weights that can be combined in different combinations to deliver the best fit. The model will take longer to learn which combinations are best as it is doing that gradeint descent process. This is basically why large neural networks can take days or weeks to train, there is just a lot of math as the data sizes and the network sizes get large.

Depending on the learning rate, and the epochs you've allocated, there is likely a point at which the model's loss starts to flatten out from epoch to epoch. This is a sign that the model has learned as much as it can, and that it is just bouncing around the minimum of the loss function. We might be able to get past this point, but it will likely involve changing something in the model's structure or configuration, not just more training. In practice, monitoring the loss as training progresses, and stopping when we reach the minimal loss is one of the primary tools to optimize training, and we have some tools that make monitoring the loss from epoch to epoch easier. 

### Complex Example - MNIST Images

Now that we are at least a little comfortable with the usage of neural networks, we can start to apply it to more useful applications, such as our old friends, the MNIST digits. Here we will make a larger model and we can also try a differnet activation function. One other difference to note is the "to_categorical" applied to the target y values. This works with the final layer of 10 output neurons - we get a softmax type (note: not actually softmax here) of output where there is a probability for being in each class. The output layer is often felxible like this in a neural network, depending on what we are doing. Here each output neuron is the probability of being in that class, we have 10, one for each digit. Our raw probabilities, which we'll print a few of below, are the probabilites of being in each class; note that those probabilities don't sum to 1, since each prediction is independent. We'd then need to add a function that is similar to "argmax", to choose the highest probability and assign a label. These details are cleaner and easier to handle with Keras, so don't get hung up on that part. With the library functions it is a bit more intuitive to deal with multiple output classes. The structure of our model will be similar to this, the middle layers will depend on whatever we add:

![MINST](images/mnist_nn.jpeg "MINST" )

Since these images, and the relationships in the data, are much more complex than the simple examples above, we should expect to see higher accuracy with more training become more visible here. More layers will also likely help, so try a few variations. In general, we can probably get really good accuracy here, it could be even better if we used more data to train, but it'll take a while. 

<b>Note:</b> one big caveat here is that we don't have a softmax activation function coded for our simple model, so we don't have any activation functions that are appropriate for this exact scenario. We are using the sigmoid activation function on the output layer, which will technically give us a multi-label classifier - one that can label things as being in more than one class at a time. If we look closely at the output percentages, they don't total to 1, like we expect in a softmax; each prediction is it's own yes/no, so a very ambiguous image could be labeled as 2+ numbers, if each of the probabilities is over .5. We could also have some digits that are not classified as anything, if they are all <.5.

In [ ]:
# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data 
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

print(x_train.shape)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 100))       
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))             # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(sigmoid, sigmoid_prime))
#net.add(FCLayer(50, 1)) 

# train on 1000 samples
# we didn't use batches, which we'll look at more next time, so we can't use too much data or it will be slow. 
net.use(mse, mse_prime)
net.fit(x_train[0:1000], y_train[0:1000], epochs=700, learning_rate=0.001)

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

### Neural Network Summary

Our simple neural network is functional, and mostly mirrors what we'll see when using real implementations with Tensorflow and Keras shortly. The key foundations of the structure of the neural networks that we need to be comfortable with are:
<ul>
<li> Layers - we'll build networks with multiple layers, including some that differ from the straightforward layers here by implementing things such as regularization. 
<li> Neurons - we need to define the number of neurons in each of the layers we create. 
    <ul>
    <li> These two things combine to define the size, or capacity, of our neural network. In general, the larger the capacity of a model, the more complex relationships it can learn, but the more likely it is to overfit, and the more training data is needed. 
    </ul>
<li> Activation Functions - we will always need at least one activation function.
<li> Loss Functions - we need to define the loss function that we will use to calculate error. 
<li> Training rounds - one of the key differences between neural networks and other models is the importance of how many rounds of training we want to run. 
    <ul>
    <li> Neural networks can often continue to learn during training for a long time, especially if we have a large capacity model with massive datasets. One of the main ways to get a good fitting model is to cut off training where we start to overfit
    <li> We can also define a stopping condition, such as a minimum error, or a minimum change in error. 
    </ul>
<li>

These key factors in neural networks generally reflect what we are used to in the previous models, but with some added complexity. The basic idea of training accurate models remains the same, but the toolset is a bit more complex and open ended. In addition, neural networks introduce a little bit of "magic" in terms of their ability to learn during training - the large capacity that many of the neural networks have to learn can allow them to "overcome/sidestep/recreate" some of the work we do in optimizing other models. Especially as the size of our data grows, neural networks can internally do things like feature selection or feature extraction without us explicitly defining those steps.